## DoublyLinkedList
---

In [8]:
use std::sync::{Arc, RwLock, Weak};
use std::borrow::{Borrow, BorrowMut};

In [9]:
#[derive(Debug)]
struct Node<T: PartialEq + PartialOrd + Clone> {
    val: T,
    next: Option<Arc<RwLock<Node<T>>>>,
    prior: Option<Weak<RwLock<Node<T>>>>,
}

In [10]:
impl<T: PartialEq + PartialOrd + Clone> Node<T> {

    fn new(val: T) -> Self {
        Node { val: val, next: None, prior: None }
    }

}

Error: duplicate definitions with name `new`

In [11]:
#[derive(Debug)]
struct LinkedList<T: PartialEq + PartialOrd + Clone> {
    head: Option<Arc<RwLock<Node<T>>>>,
    tail: Option<Weak<RwLock<Node<T>>>>,
}

In [12]:
impl<T: PartialEq + PartialOrd + Clone> LinkedList<T> {

    fn new() -> Self {
        LinkedList { head: None, tail: None }
    }

    fn push(&mut self, val: T) -> &Self {
        let mut v1 = Arc::new(RwLock::new(Node::new(val)));
        if self.head.is_none() {
            self.head = Some(v1.clone());
            self.tail = Some(Arc::downgrade(&v1));
        } else {
            let mut v2 = self.head.take().unwrap();
            v1.borrow_mut().write().unwrap().next = Some(v2.clone());
            v2.borrow_mut().write().unwrap().prior = Some(Arc::downgrade(&v1));
            self.head = Some(v1);
        }
        self
    }
    
    fn push_back(&mut self, val: T) -> &Self {
        let mut v2 = Arc::new(RwLock::new(Node::new(val)));
        if self.head.is_none() {
            self.head = Some(v2.clone());
            self.tail = Some(Arc::downgrade(&v2));
        } else {
            let mut v1 = self.tail.take().unwrap().upgrade().unwrap();
            v1.borrow_mut().write().unwrap().next = Some(v2.clone());
            v2.borrow_mut().write().unwrap().prior = Some(Arc::downgrade(&v1));
            self.tail = Some(Arc::downgrade(&v2));
        }
        self
    }
    
    fn clear(&mut self) {
        self.head = None;
        self.tail = None;
    }
    
    fn len(&self) -> usize {
        let mut l = 0usize;
        if self.head.is_none() {
            return l;
        }
        let mut current = self.head.as_ref().unwrap().clone();
        loop {
            l += 1;
            if current.read().unwrap().next.is_none() {
                return l;
            }
            let next = current.read().unwrap().next.as_ref().unwrap().clone();
            current = next;
        }
    }

}

In [13]:
let mut q = LinkedList::<i32>::new();
q

LinkedList { head: None, tail: None }

In [14]:
q.push(8)

LinkedList { head: Some(RwLock { data: Node { val: 8, next: None, prior: None }, poisoned: false, .. }), tail: Some((Weak)) }

In [15]:
q.push(6)

LinkedList { head: Some(RwLock { data: Node { val: 6, next: Some(RwLock { data: Node { val: 8, next: None, prior: Some((Weak)) }, poisoned: false, .. }), prior: None }, poisoned: false, .. }), tail: Some((Weak)) }

In [16]:
q.push(1)

LinkedList { head: Some(RwLock { data: Node { val: 1, next: Some(RwLock { data: Node { val: 6, next: Some(RwLock { data: Node { val: 8, next: None, prior: Some((Weak)) }, poisoned: false, .. }), prior: Some((Weak)) }, poisoned: false, .. }), prior: None }, poisoned: false, .. }), tail: Some((Weak)) }

In [17]:
let mut q2 = LinkedList::<&str>::new();
q2

LinkedList { head: None, tail: None }

In [18]:
q2.push_back("Joel")

LinkedList { head: Some(RwLock { data: Node { val: "Joel", next: None, prior: None }, poisoned: false, .. }), tail: Some((Weak)) }

In [19]:
q2.push_back("was")

LinkedList { head: Some(RwLock { data: Node { val: "Joel", next: Some(RwLock { data: Node { val: "was", next: None, prior: Some((Weak)) }, poisoned: false, .. }), prior: None }, poisoned: false, .. }), tail: Some((Weak)) }

In [20]:
q2.push_back("here.")

LinkedList { head: Some(RwLock { data: Node { val: "Joel", next: Some(RwLock { data: Node { val: "was", next: Some(RwLock { data: Node { val: "here.", next: None, prior: Some((Weak)) }, poisoned: false, .. }), prior: Some((Weak)) }, poisoned: false, .. }), prior: None }, poisoned: false, .. }), tail: Some((Weak)) }

In [35]:
q.head.as_ref().unwrap().read().unwrap().val

1